<a href="https://colab.research.google.com/github/Trantracy/Analyze-music-company-by-SQL/blob/master/Advanced_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://i.imgur.com/0AUxkXt.png)

# MLE - Advanced SQL

### Connect to the database

![](https://i.imgur.com/kCaiMK5.png)

**Run those cells below**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sqlite3
import pandas as pd

# Replace the path below with yours
conn = sqlite3.connect('gdrive/My Drive/FTMLE - Tonga/Data/chinook.db')

### Sub-queries
Sub-queries are SQL queries nested inside a larger query.
```sql
SELECT column_names
FROM table_names
WHERE value IN
  (SELECT column_names
  FROM other_table
  WHERE condition)
```

In [0]:
query = """
        
        WITH Total_income as 
          (SELECT a.name, sum(i.UnitPrice * i.Quantity) as Total
          FROM artists as a
          JOIN albums as al ON a.ArtistId = al.ArtistId
          JOIN tracks as t ON al.albumId = t.albumId
          JOIN invoice_items as i ON t.TrackId = i.TrackId
          GROUP BY a.name)

        SELECT * FROM Total_income    
        WHERE Total > 
        (SELECT avg(Total) FROM Total_income)
        ORDER BY Total DESC;
        """
pd.read_sql_query(query,conn)

,name,Total
0,Iron Maiden,138.60
1,U2,105.93
2,Metallica,90.09
3,Led Zeppelin,86.13
4,Lost,81.59
5,The Office,49.75
6,Os Paralamas Do Sucesso,44.55
7,Deep Purple,43.56
8,Faith No More,41.58
9,Eric Clapton,39.60


In [0]:
# List all the artist whose income is higher than the average income 
query = '''
           WITH artist_income AS 
            (SELECT ar.Name, SUM(i.UnitPrice * i.Quantity) AS TotalIncome
              FROM artists AS ar 
                  JOIN albums AS al ON ar.ArtistId = al.ArtistId
                  JOIN tracks AS t ON t.AlbumId = al.AlbumId
                  JOIN invoice_items as i ON t.TrackId = i.TrackId
              GROUP BY ar.Name)
            SELECT *
            FROM artist_income
            WHERE TotalIncome > 
              (SELECT AVG(TotalIncome) FROM artist_income)
            ORDER BY TotalIncome DESC;
'''

pd.read_sql_query(query, conn)

In [0]:
query = """Select t.Name as Track_name, g.Name as genrename
       FROM genres as g
        JOIN tracks as t ON g.GenreID = t.GenreID  
        GROUP BY t.GenreID """
pd.read_sql_query(query,conn)

,Track_name,genrename
0,Love Comes,Rock
1,OAM's Blues,Jazz
2,Sweet Lady Luck,Metal
3,Senhor Delegado/Eu Não Aguento,Alternative & Punk
4,20 Flight Rock,Rock And Roll
5,Remedy,Blues
6,Muita Bobeira,Latin
7,Until My Dying Day,Reggae
8,Amy Amy Amy (Outro),Pop
9,Koyaanisqatsi,Soundtrack


In [0]:
# List all the tracks that has file size greater and equal than average file size of their corresponding genre 

query = '''
            WITH tracks_genres AS 
              (SELECT t.Name, t.Bytes, g.Name as Genre
              FROM tracks AS t
              JOIN genres AS g
              ON g.GenreId = t.GenreId)
              
            SELECT tg.Name, tg.Bytes, tg.Genre
            FROM tracks_genres as tg
            JOIN 
              (SELECT Genre, ROUND(AVG(Bytes), 2) AS AvgBytes
              FROM tracks_genres
              GROUP BY Genre) AS ag
            ON ag.Genre = tg.Genre
            WHERE Bytes >= 
              (SELECT AVG(Bytes)
              FROM tracks_genres
              WHERE Genre = tg.Genre)
             
'''

pd.read_sql_query(query, conn)

,Name,Bytes,Genre
0,For Those About To Rock (We Salute You),11170334,Rock
1,Go Down,10847611,Rock
2,Let There Be Rock,12021261,Rock
3,Problem Child,10617116,Rock
4,Overdose,12066294,Rock
...,...,...,...
1492,"3 Gymnopédies: No.1 - Lent Et Grave, No.3 - Le...",6458501,Classical
1493,Symphony No. 2: III. Allegro vivace,6129146,Classical
1494,"Metopes, Op. 29: Calypso",5548755,Classical
1495,"Concerto for Violin, Strings and Continuo in G...",16454937,Classical


### Conditionals
The If-else statements in SQL
```sql
CASE
  WHEN condition1 THEN result1
  WHEN condition2 THEN result2
  WHEN condition3 THEN result3
  ELSE result4
END;
```

In [0]:
# Validate the result of the previous query 

query = '''
           SELECT Validation, COUNT(Validation) AS Count FROM
            (WITH tracks_genres AS 
              (SELECT t.Name, t.Bytes, g.Name as Genre
              FROM tracks AS t
              JOIN genres AS g
              ON g.GenreId = t.GenreId)
            SELECT tg.Name, tg.Bytes, tg.Genre, 
            CASE 
              WHEN tg.Bytes - ag.AvgBytes >= 0 THEN "Correct"
              ELSE "Error"
            END AS Validation
            FROM tracks_genres as tg
            JOIN 
              (SELECT Genre, ROUND(AVG(Bytes), 2) AS AvgBytes
              FROM tracks_genres
              GROUP BY Genre) AS ag
            ON ag.Genre = tg.Genre
            WHERE Bytes >= 
              (SELECT AVG(Bytes)
              FROM tracks_genres
              WHERE Genre = tg.Genre))
          GROUP BY Validation;
             
'''

pd.read_sql_query(query, conn)

,Validation,Count
0,Correct,1497


### Set Operators
Set Operators allow the results multiple queries to be combined together.
```sql
SELECT col1, col2
FROM table1
UNION / UNION ALL / INTERSECT / EXCEPT
SELECT col1, col2
FROM table2
```

In [0]:
# List all customers and employees coming from Canada

query = '''
    SELECT FirstName, LastName
    FROM customers 
    WHERE country ="Canada"
    UNION 
    SELECT FirstName, LastName
    FROM employees
    WHERE country ="Canada";


             
'''

pd.read_sql_query(query, conn)

In [0]:
# List all customers and employees coming from Canada

query = '''
          SELECT LastName, FirstName, Country FROM customers
          WHERE Country = "Canada"
          UNION ALL
          SELECT LastName, FirstName, Country FROM employees
          WHERE Country = "Canada";
             
'''

pd.read_sql_query(query, conn)

,LastName,FirstName,Country
0,Tremblay,François,Canada
1,Philips,Mark,Canada
2,Peterson,Jennifer,Canada
3,Brown,Robert,Canada
4,Francis,Edward,Canada
5,Silk,Martha,Canada
6,Mitchell,Aaron,Canada
7,Sullivan,Ellie,Canada
8,Adams,Andrew,Canada
9,Edwards,Nancy,Canada


In [0]:
query = '''
 SELECT *
 FROM invoices 
 LIMIT 2;    
'''

pd.read_sql_query(query, conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96


In [0]:
# Query the best- and worst-performing 
# Sales Representative

query = '''
WITH saler AS
 (SELECT e.FirstName as saler, sum(i.Total) as sale
 FROM employees as e 
 JOIN customers as c ON e.EmployeeId = c.SupportRepId
 JOIN invoices as i ON c.CustomerId  = i.CustomerId
 GROUP BY e.EmployeeId)

 SELECT saler, max(sale) as sale
 FROM saler 
 UNION 
 SELECT saler, min(sale) as sale
 FROM saler

'''

pd.read_sql_query(query, conn)

,saler,sale
0,Jane,833.04
1,Steve,720.16


In [0]:
# Query the best- and worst-performing Sales Representative

query = '''
          WITH employee_sales AS 
            (SELECT e.FirstName, e.LastName, SUM(Total) AS Revenue
            FROM invoices AS i 
              JOIN customers AS c ON i.CustomerId = c.CustomerId
              JOIN employees as e ON c.SupportRepId = e.EmployeeId
            GROUP BY e.EmployeeId)
          SELECT * FROM 
            (SELECT * 
            FROM employee_sales
            ORDER BY Revenue DESC
            LIMIT 1)
          UNION ALL
          SELECT * FROM
            (SELECT * 
            FROM employee_sales
            ORDER BY Revenue ASC
            LIMIT 1)  

'''

pd.read_sql_query(query, conn)

,FirstName,LastName,Revenue
0,Jane,Peacock,833.04
1,Steve,Johnson,720.16


In [0]:
# Query the Sales Representative earning highest revenue and one making most deals
# Return only one row if it is the same person

query = '''
          WITH employee_sales AS 
            (SELECT e.FirstName, e.LastName, SUM(Total) AS Revenue, COUNT(Total) AS Deals
            FROM invoices AS i 
              JOIN customers AS c ON i.CustomerId = c.CustomerId
              JOIN employees as e ON c.SupportRepId = e.EmployeeId
            GROUP BY e.EmployeeId)
          SELECT * FROM 
            (SELECT * 
            FROM employee_sales
            ORDER BY Revenue DESC
            LIMIT 1)
          UNION
          SELECT * FROM
            (SELECT * 
            FROM employee_sales
            ORDER BY Deals DESC
            LIMIT 1)  

'''

pd.read_sql_query(query, conn)

,FirstName,LastName,Revenue,Deals
0,Jane,Peacock,833.04,146


In [0]:
# Query the Sales Representative earning highest revenue and one making most deals
# Return only one row if it is the same person

query = '''
WITH saler AS
 (SELECT e.FirstName as saler, sum(i.Total) as sale
 FROM employees as e 
 JOIN customers as c ON e.EmployeeId = c.SupportRepId
 JOIN invoices as i ON c.CustomerId  = i.CustomerId
 GROUP BY e.EmployeeId)

 SELECT saler, max(sale) as sale
 FROM saler 
'''

pd.read_sql_query(query, conn)

In [0]:
query = '''
SELECT *
FROM invoices
LIMIT 2; 
'''

pd.read_sql_query(query, conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96


In [0]:
query = '''
SELECT *
FROM customers
LIMIT 2; 
'''

pd.read_sql_query(query, conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
